# How to generate vector field training sets

This notebook explains how to use the components of the `vectorizor` module to
generate 3d vector fields from 3d label volumes.

# Parsing track data

Let's assume track analysis was derived in MatLab and dumped to a JSON format similar
to JSON format to a file that looks like the following:

In [1]:
file_name = "Gata6Nanog1.json"
import json
json_graph = json.load(open(file_name))

In [2]:
#json_graph

# Getting track mappings for the timestamps

The vector generation process uses the concept of tracks across timestampe.
These tracks need to be identified using preprocessing as follows.

The following function will identify "tracks" connecting related labels in the JSON data:

In [3]:
from mouse_embryo_labeller.vectorizor import make_tracks_from_haydens_json_graph

timestamp_mapping = make_tracks_from_haydens_json_graph(json_graph)

In [4]:
ts6m = timestamp_mapping[6]
ts6m

{5: 1, 3: 2, 2: 3, 6: 4, 4: 5, 7: 6, 8: 7, 9: 8, 1: 9}

In [5]:
ts7m = timestamp_mapping[7]
ts7m

{5: 1, 1: 2, 3: 3, 6: 4, 9: 5, 7: 6, 2: 7, 8: 8, 4: 9}

The 2 cells above show the track mappings for timestamps 6 and 7.

The mappings indicate, for example, that in timestamp 6 label 9 maps to track 8,
but in timestamp 7 label 8 maps to track 8.  So these two timestamps are talking about
the same nucleus using different label numbers.

These mappings are required for generating vector fields from label volume arrays.

For illustration purposes let's make some fake volume arrays for timestamps 6 and 7 using
inverse mappings:

In [6]:
import numpy as np

inv6 = {track: label for (label, track) in ts6m.items()}
inv7 = {track: label for (label, track) in ts7m.items()}

N = 10
A = np.zeros((N,N,N), dtype=np.int)
B = np.zeros((N,N,N), dtype=np.int)

A[2:5, 1:4, 3:6] = inv6[3]
B[4:7, 3:6, 7:9] = inv7[3]

A[5:7, 7:9, 0:2] = inv6[1]
B[7:9, 4:7, 3:6] = inv7[1]

if 0:
    A[0,0,0] = inv6[4]
    B[9,2,9] = inv7[4]

    A[1:3,2:5,1:3] = inv6[5]
    B[8:10,9,8:10] = inv7[5]

labels6 = A
labels7 = B

# Generating the vectors in one step:

In [7]:
from mouse_embryo_labeller.vectorizor import get_track_vector_field

vector_field = get_track_vector_field(
    old_label_array=labels6, 
    old_labels_to_tracks=ts6m,
    new_label_array=labels7,
    new_labels_to_tracks=ts7m,
    di = (10, 0, 0),  #  xyz offset between A[i,j,k] and A[i+1,j,k]
    dj = (0, 10, 0),  #  xyz offset between A[i,j,k] and A[i,j+1,k]
    dk = (0, 0, 10),  #  xyz offset between A[i,j,k] and A[i,j,k+1]
    )
vector_field.shape

(10, 10, 10, 3)

# What is it doing?

The following breaks down the steps for making the vector field and includes
a small scale 3d illustration of the output.

# First replace labels with track numbers in the arrays

In [8]:
from mouse_embryo_labeller.vectorizor import unify_tracks

(tracks6, tracks7) = unify_tracks(
    A=labels6,  # label array
    A_label_2_track=ts6m,   # mapping of labels in A to track numbers
    B=labels7,  # label array
    B_label_2_track=ts7m,   # mapping of labels in A to track numbers
)

# Then the VectorMaker makes the vectors connecting tracks

In [9]:
from mouse_embryo_labeller.vectorizor import VectorMaker

V = VectorMaker(tracks6, tracks7)
V.scaled_vectors.shape

(10, 10, 10, 3)

# For small test cases you can view the vectors in a widget

In [ ]:
W = V.widget()